In [1]:
import pandas as pd
import csv, re, os

In [2]:
origin = "./"        

In [3]:
errorFrequencyDF = pd.DataFrame(columns = ['DateTime', 'filename', 'Tag_name', 'Error', 'timed out',
       'fail', 'Failed', 'Stopped', 'PERM', 'PERF', 'INFO', 'TEMP', 'UNKN',
       'PEND', 'failure', 'fault', 'Err', 'not responding'])
for direc in os.listdir(origin):
    if os.path.isdir(direc) and direc.endswith('hotWordsCount'):
        for file in os.listdir(direc):
            tempdf = pd.read_csv(origin+direc+'/'+file)
            if 'Unnamed: 0' in tempdf.columns:
                tempdf.drop(labels='Unnamed: 0',axis=1,inplace=True)
            errorFrequencyDF = errorFrequencyDF.append(tempdf,ignore_index=True)
errorFrequencyDF.to_csv("HotWordsCount.csv",index=False)

In [4]:
df = pd.read_csv("HotWordsCount.csv")
# df.drop(labels='Unnamed: 0',axis=1,inplace=True)

In [5]:
df['Date'] = df['DateTime'].apply(lambda r: r.split('T')[0])
df['Time'] = df['DateTime'].apply(lambda r: r.split('T')[1])

In [6]:
# print(df.columns)
df['Month'] = df['Date'].apply(lambda r: r.split('-')[1])
df['Year'] = df['Date'].apply(lambda r: r.split('-')[0])
df = df.loc[:,['Date', 'Time','Year','Month','filename', 'Tag_name', 'Error', 'timed out', 'fail',
       'Failed', 'Stopped', 'PERM', 'PERF', 'INFO', 'TEMP', 'UNKN', 'PEND',
       'failure', 'fault', 'Err', 'not responding']]
print(df.head())

         Date            Time  Year Month filename    Tag_name  Error  \
0  2017-07-01  23:59:55-04:00  2017    07     r012     systemd      0   
1  2017-07-01  23:59:55-04:00  2017    07     r012       CROND      0   
2  2017-07-01  23:59:55-04:00  2017    07     r012  run-parts(      0   
3  2017-07-01  23:59:55-04:00  2017    07     r012     anacron      0   
4  2017-07-01  23:59:55-04:00  2017    07     r012        sshd      0   

   timed out  fail  Failed       ...        PERM  PERF  INFO  TEMP  UNKN  \
0          0     0       0       ...          96     0     0     2     0   
1          0     0       0       ...           0     0     0     0     0   
2          0     0       0       ...           0     0     0     0     0   
3          0     0       0       ...           0     0     0     0     0   
4          0     0       0       ...           0     0     0     0     0   

   PEND  failure  fault  Err  not responding  
0     0        0      0    0               0  
1     0   

In [15]:
df['YearMonth'] = df.apply(lambda r: r['Year']+'-'+r['Month'], axis=1)
df_r = df[df['filename']>'br018']
df_br = df[df['filename']<'r001']

Index(['Date', 'Time', 'Year', 'Month', 'filename', 'Tag_name', 'Error',
       'timed out', 'fail', 'Failed', 'Stopped', 'PERM', 'PERF', 'INFO',
       'TEMP', 'UNKN', 'PEND', 'failure', 'fault', 'Err', 'not responding'],
      dtype='object')


In [16]:
df.head()

,Date,Time,Year,Month,filename,Tag_name,Error,timed out,fail,Failed,...,PERF,INFO,TEMP,UNKN,PEND,failure,fault,Err,not responding,YearMonth
0,2017-07-01,23:59:55-04:00,2017,07,r012,systemd,0,0,0,0,...,0,0,2,0,0,0,0,0,0,2017-07
1,2017-07-01,23:59:55-04:00,2017,07,r012,CROND,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017-07
2,2017-07-01,23:59:55-04:00,2017,07,r012,run-parts(,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017-07
3,2017-07-01,23:59:55-04:00,2017,07,r012,anacron,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017-07
4,2017-07-01,23:59:55-04:00,2017,07,r012,sshd,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2017-07


In [17]:
groups = df_r.groupby(by=['YearMonth','Tag_name'])
print(groups)

In [18]:
meanDF = groups.mean().reset_index()

In [22]:
months = pd.unique(meanDF['YearMonth'])

In [25]:
anomalyMonth = []
count = 1
for month in months:
    compareDF = meanDF[meanDF['YearMonth']<month]
    if len(compareDF)>0:
        tag_avg_values = compareDF.groupby(['Tag_name']).mean().reset_index()
        tag_avg_values['sumCompare'] = tag_avg_values.sum(axis=1)
        monthDF = meanDF[meanDF['YearMonth']==month]
        monthDF['sumMonth'] = monthDF.sum(axis=1)
        resultDF = pd.merge(tag_avg_values.loc[:,['Tag_name','sumCompare']],monthDF.loc[:,['Tag_name','sumMonth']],on=['Tag_name'],how='outer')
        resultDF.to_csv("Compare "+str(count)+".csv")
        count+=1
        print(month)
        for row in resultDF['Tag_name']:
            tagDF = resultDF[resultDF['Tag_name']==row]
            if tagDF['sumCompare'].values[0] and tagDF['sumMonth'].values[0]:
                if abs(tagDF['sumCompare'].values[0] - tagDF['sumMonth'].values[0])>0.1:
                    anomalyMonth.append(str(month)+' '+str(row))
print(anomalyMonth)
                
    

/opt/packages/anaconda3.5/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


2017-08
2017-10
2017-11
2018-02
2018-03
['2017-08 automount', '2017-08 crond', '2017-08 kernel', '2017-08 nslcd', '2017-08 puppet-agent', '2017-08 sendmail', '2017-08 slurmd-epilog', '2017-08 slurmd-prolog', '2017-08 systemd', '2017-10 automount', '2017-10 crond', '2017-10 kernel', '2017-10 ldconfig', '2017-10 logger', '2017-10 network', '2017-10 nhc', '2017-10 nslcd', '2017-10 puppet-agent', '2017-10 root', '2017-10 slurmd-epilog', '2017-10 slurmd-prolog', '2017-10 sshd', '2017-10 sudo', '2017-10 systemd', '2017-11 crond', '2017-11 kernel', '2017-11 nslcd', '2017-11 puppet-agent', '2017-11 root', '2017-11 slurmd-epilog', '2017-11 slurmd-prolog', '2017-11 systemd', '2018-02 avahi-daemon', '2018-02 crond', '2018-02 journal', '2018-02 kernel', '2018-02 ldconfig', '2018-02 network', '2018-02 nhc', '2018-02 nslcd', '2018-02 opa-init-kernel', '2018-02 pam_slurm', '2018-02 puppet', '2018-02 puppet-agent', '2018-02 root', '2018-02 slurmd-epilog', '2018-02 slurmd-prolog', '2018-02 systemd', '2